##  텍스트 처리용 IMDb 영화 리뷰 데이터 준비
        o 감성분석( = 의견분석)은 문서 성향을 분석하는 것을 주로 함
        o 이 장에서 사용될 영화 리뷰 데이터의 긍정은 별점 6개 이상으로 설정함
                부정은 그 나머지를 뜻함
                
### 영화 리뷰 데이터셋 구하기
        pyprind : 진행상황을 보기 위한 모듈
                - Progbar : 읽어드릴 문서 개수와 같게 함 

In [5]:
#!pip install pyprind
import pyprind
import pandas as pd
import os

# `basepath`를 압축 해제된 영화 리뷰 데이터셋이 있는
# 디렉토리로 바꾸세요

basepath = 'C:\\Users\\jye12\\Python\\file\\pytest_대용량\\aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 
                      'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], 
                           ignore_index=True)
            pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:29


합친 데이터셋의 클래스 레이블 순서대로 나열되어 있으므로 np.random 모듈의 permutation 함수를 사용하여 데이터프레임을 섞음  
나중에 로컬 디스크에서 데이터를 조금씩 읽어 훈련 세트와 테스트 세트를 나눌 때 좋음  
                
편의상 여기서 합쳐서 섞은 영화 리뷰 데이터셋을 csv 파일로 저장

In [10]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

df.to_csv('C:\\Users\\jye12\\Python\\file\\pytest_대용량/movie_data.csv', index=False, encoding='utf-8')

올바르게 저장되었는지 확인

In [11]:


df = pd.read_csv('C:\\Users\\jye12\\Python\\file\\pytest_대용량/movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"Election is a Chinese mob movie, or triads in ...",1
1,I was just watching a Forensic Files marathon ...,0
2,Police Story is a stunning series of set piece...,1


데이터 프레임의 shape 확인

In [12]:
df.shape

(50000, 2)

### Bow 모델 소개
        o 텍스트나 단어 같은 범주형 데이터를 머신 러닝에 주입하기 전에 수치형 형태로 변환 필요
        o 이 절에서는 텍스트를 수치 특성 벡터로 표현하는 BoW(Bag-of -Word)를 소개
        o Bow
                1. 전체 문서에 대해 고유한 토큰(token), 예를 들어 단어로 이루어진 어휘 사전을 만듬
                2. 특정 문서에 각 단어가 얼마나 자주 등장하는지 헤아려 문서의 특성 벡터를 만듬
                
                각 문서에 있는 고유한 단어는 BoW 어휘 사전에 있는 모든 단어의 일부분이므로 특성 벡터는 대부분 0 -> 이 특성 벡터를 희소하다고 함
### 단어를 특성 벡터로 변환 (DTM?)
        o Scikit-learn 의 CountVectorizer : 문서 또는 문장으로 이루어진 텍스트 데이터 배열을 입력받아 Bow 모델을 만듬
                CountVectorizer의 fir_transform 메서드를 호출하면 Bow 모델의 어휘 사전을 구축하고 다음 세 문장을 희소한 특성 벡터로 변환

In [13]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

내용물 확인해보면 이해에 도움

In [14]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


이전 결과에서 볼 수 있듯이 어휘 사전은 고유 단어와 정수 인덱스가 매핌된 파이썬 딕셔너리에 저장되어 있음  
그 다음 만들어진 특성 벡터를 출력해보자

In [16]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


특성 벡터의 이런 값들을 단어 빈도(term frequency)라고 합니다. 문서 d에 등장한 단어 t의 횟수를 tf(t,d)처럼 같이 씁니다.

### tf - idf 를 사용하여 단어 적합성 평가
        o 일반적으로 자주 등장하는 단어는 유용하거나 판별에 필요한 정보를 가지고 있지 않습니다.
        o 이 절에서는 특성 벡터에서 자주 등장하는 단어의 가중치를 낮추는 기법인 tf-idf(term frequency - inverse document frequency)를 배우겠습니다. 
                td-idf는 단어 빈도(tf)와 역문서 빈도(idf)의 곱으로 정의됨
        
                - tf 는 위에서 정의한 것
                - idf 는 log [ (전체 문서 개수) / ( 1 + df(d,t)) ]
                        df(d,t)는 단어 t가 포함된 d의 갯수
                        log를 취한 이유는 df값이 작을 때 idf값이 너무 커지지 않게 하기 위함
                        
Scickit-learn에는 CountVectorizer 클래스를 입력 받아 tf-idf로 변환하는 TfidTransformer 클래스가 구현되어 있습니다.

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer
np.set_printoptions(precision=2)
tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


### 텍스트 데이터 정제


In [23]:
df.loc[2,'review']

"Police Story is a stunning series of set pieces for Jackie Chan to show his unique talents and bravery. Some of the stunts here are among Jackie's best and most dangerous the whole mall fight finale is probably Jackie's greatest single fight sequence, more brutal and less comedic then say Project A or Drunken Master at the end of the fight you can almost feel the pain of the impact.<br /><br />But unfortunately the rest of the film doesn't hold up to this quality as it is a rather formulaic cop thriller with some comedy elements. I always prefer JC in films such as Project A where his natural comic talents shine through. In the serious confines of some elements of Police Story it just doesn't work for me. Having said that though this is still up there with Jackies best films due to the incredible stunt work and sheer spectacle.<br /><br />As usual with Hong Kong films avoid the English dubbed DVD version as it is truly awful stick with the subtitles.<br /><br />Great stunts, OK movie 